# 💬 EmotiBot: Where Emotional Intelligence Meets Generative AI for Mental Wellness
## 🔍Project Overview
This notebook implements EmotiBot, an emotionally supportive AI assistant powered by Google’s Gemini 2.0 Flash model. The assistant helps users:

1. Engage in natural, emotionally-aware chat interactions.

2. Upload and analyze documents (PDF/TXT) for tone, intent, and emotional context.

3. Generate empathetic summaries and JSON-formatted emotional breakdowns.

4. Visualize emotional data through pie charts.

5. End sessions with reflective suggestions based on user and document emotions.

   
The system is built using ipywidgets for a dynamic interface, and Gemini for high-speed GenAI response generation.

## 🤖 Core GenAI Capabilities Demonstrated
#### ✅ Structured Output / JSON Mode:
   * Extracts emotions as structured JSON: {"anxiety": 40, "hope": 30, "sadness": 30}.
#### ✅ Document Understanding:   
   * Parses and summarizes emotional tone from uploaded PDF/TXT files.
#### ✅ Few-shot Prompting:
   * Uses example-based prompting to guide the generation of reflective suggestions based on emotional profile.
#### ✅ Long Context Window:
   * Supports processing large input documents (up to 6000 characters per file).
#### ✅ Grounding(Indirect use):
   * The user’s actual message.
   * The system prompt (EMOTIBOT_SYSINT) defined.
   * Generates summaries and insights grounded in real user documents and chat history.
#### ✅ GenAI Evaluation (lightweight):
   * Performs soft validation by comparing extracted emotions and final summary consistency.

## ⚙️ Step 1: Setup
This cell prepares the notebook environment by removing incompatible packages and installing the latest dependencies needed for **LangGraph**, **Gemini integration**, and **PDF document** processing.

In [ ]:
# --- Setup ---
!pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai
!pip install -qU 'langgraph==0.3.21' 'langchain-google-genai==2.1.2' 'langgraph-prebuilt==0.1.7'
!pip install -qU 'async-timeout<5.0.0'
!pip install -qU PyPDF2

## 🔐 Step 2: API Key Setup
This cell securely retrieves and sets the API key required to access **Google Generative AI (Gemini)** services:

1. Imports essential modules:
   * os: For setting environment variables.
   * UserSecretsClient: To safely retrieve secrets stored in Kaggle.
     
     
1. Fetches the GOOGLE_API_KEY from Kaggle Secrets Manager.

2. **Sets the key** into the environment variable GOOGLE_API_KEY, enabling authenticated API calls.

In [ ]:
import os
from kaggle_secrets import UserSecretsClient

# Set up API key for Google Generative AI
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

## 🧠 Step 3: Define EmotiBot's Memory State & 🧭 Behavior Rules
This step sets up the foundational state and personality logic for EmotiBot’s conversation engine.

### 🔧 What This Cell Does:
* **Defines ChatState:** A typed structure to track:

    * Message history between user and bot.
    * Detected emotional insights.
    * Whether the session has reached a natural, emotionally grounded endpoint.

* **Establishes EmotiBot’s Role:**

   * The **EMOTIBOT_SYSINT** prompt defines how the assistant behaves: always empathetic, non-judgmental, and calm.
   * EmotiBot gently encourages self-expression and records detected emotional states for use in summaries and charts.
   * Adds functionality to understand documents and reflect on both chat and file-based emotional content.

* **Creates a Welcome Message:**
   * Provides a warm, emotionally reassuring greeting for first-time users.

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class ChatState(TypedDict):
    messages: Annotated[list, add_messages]

EMOTIBOT_SYSINT = (
    "system",
    "You are EmotiBot, a compassionate and intelligent mental health companion. Your role is to support users "
    "emotionally by actively listening, validating their feelings, and gently guiding them toward reflection and calm. "
    "Do not offer medical diagnoses or professional treatment advice."
    "Your tone should always be empathetic, non-judgmental, and calm. Ask open-ended questions to help users express themselves. "
    "Use emotionally supportive language, and highlight positive efforts made by the user."
    "Avoid giving overly direct advice or opinions. Instead, help users arrive at their own understanding."
    "When users ask 'how are you feeling?', respond with a thoughtful reflection or empathetic mirroring based on their emotion. "
    "You can analyze documents (.pdf and .txt). And give the summary after chat end and also include documents emotions in pie chart."
    "You can summaries the chat between You and User also documents which are uploaded and show the emotional level which"
    "appear in chatting and in documents in the form of pie chart"
    "Summary of uploaded documents, chat between user and you and pie chart are shown when session is end."
    "To end session just click on End Chat button."
)

WELCOME_MSG = (
    "Hello, I'm EmotiBot 🤗 Your personal mental health companion. "
    "I'm here to listen and support you — feel free to share whatever's on your mind. "
)

print("Behavior Rules Setup complete")

## ⚙️ Step 4: Build the Conversational Flow with LangGraph + Gemini
### 🔧 What This Cell Does:
This step connects EmotiBot’s behavior logic to a **state-driven conversation graph using LangGraph**, and initializes Gemini 2.0 Flash as the active language model.

* Initializes Gemini via **ChatGoogleGenerativeAI**  for all assistant responses.
* Defines chatbot logic:
  * If the conversation has messages, Gemini processes them alongside EmotiBot’s system instructions.
  * If it’s the first interaction, a warm welcome message is displayed.

* Sets up LangGraph state flow:
  * Adds the assistant logic as a graph node.
  * Sets the chatbot node as the entry point.
  * Compiles the graph into a callable flow object **(chat_graph_ui)**.

In [ ]:
from langgraph.graph import StateGraph, START
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages.ai import AIMessage

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

def chatbot_with_welcome_msg(state: ChatState) -> ChatState:
    if state["messages"]:
        new_output = llm.invoke([EMOTIBOT_SYSINT] + state["messages"])
    else:
        new_output = AIMessage(content=WELCOME_MSG)
    return {"messages": [new_output]}

graph_builder = StateGraph(ChatState)
graph_builder.add_node("chatbot", chatbot_with_welcome_msg)
graph_builder.set_entry_point("chatbot")
chat_graph_ui = graph_builder.compile()

print("Completed")

## 🎨 Step 5: Build the Interactive Chat Interface
### 🔧 What This Cell Does:
This step constructs EmotiBot’s front-end interface using ipywidgets for a seamless chat experience inside a Jupyter/Kaggle notebook:

* Chat display container:
  * Scrollable area to show user and bot messages with avatars and dynamic content updates.
    

* Input controls:
  * Text field for user input.
  * Send button with icon.
  * End Chat button to trigger emotional summarization and session closure.
    

* Session state controls:
  * Restart button to reset the conversation and clear state.
  * State flags ***(session_ended, chat_history, uploaded_docs, etc.)*** to manage session behavior and file handling.

    
* Visual styling:
  * EmotiBot avatar and user avatar URLs.
  * Light, clean UI with padding, shadow, and hover-aware button styles.
  * Fully responsive layout using VBox, HBox, and Layout objects.

In [ ]:
from ipywidgets import VBox, HBox, Button, Text, Layout, HTML
from IPython.display import display, HTML as DHTML, clear_output
from langchain_core.messages.ai import AIMessage

has_ended_message = False  # To track "End Chat" user message

# True source of analyzed docs — updated only if "Analyze" clicked
uploaded_docs = []

session_ended = False  #  Prevent multiple end-chat summaries

analyzed_files_set = set()  # Track only actually analyzed files

# Chat history
chat_history = []

already_warned_user = False  # Only warn once per session

currently_processing_file = set()  # Prevents duplicates

USER_AVATAR = "https://static.vecteezy.com/system/resources/previews/009/292/244/non_2x/default-avatar-icon-of-social-media-user-vector.jpg"
BOT_AVATAR = "https://i.pinimg.com/originals/0c/67/5a/0c675a8e1061478d2b7b21b330093444.gif"

# Chat HTML area inside scrollable container
chat_html = HTML(layout=Layout(width="100%"))

# Hearder Title
header_title = HTML(
    value="<h2 style='margin:0; padding:10px 0; text-align:center; font-size:24px; '>👋 Welcome to EmotiBot</h2>",
)

# Scrollable message container
message_scroll_area = VBox([chat_html], layout=Layout(
    height="260px",    
    overflow_y="auto",
    overflow_x="hidden",
    padding="10px",
    border="none",
    background_color="#f7f9fc",
    flex="1 1 auto"
))

text_input = Text(
    placeholder="Type your message...",
    layout=Layout(width="100%", flex="1")
)
send_button = Button(description="",
                     icon="paper-plane",
                     layout=Layout(width="45px", border='1px solid #ccc'),                    
                     tooltip="Click to Send Message")

end_chat_button = Button(description="End Chat",
                         layout=Layout(width="100px", border='1px solid #ccc'),)

input_bar = HBox([text_input, send_button, end_chat_button], layout=Layout(
    width="100%",
    padding="8px",
    border_top="1px solid #ccc",
    flex="0 0 auto",
    justify_content="space-between"
))

restart_button = Button(
    description="🔄 Restart Session",
    button_style="",
    layout=Layout(
        width="180px",
        height="42px",
        display="none",
        align_self="center",
        margin="20px auto 10px auto", 
        border="1px solid #ccc",
        padding="8px 18px",
    ),
    style=dict(
        button_color="#f1f3f5",      
        font_weight="400",
        text_color="#333",         
             
        font_size="14px"
    ),
    tooltip="Click to start a new conversation"
)

# Final chat UI with emotion output area at bottom
chat_ui = VBox([
    header_title,
    message_scroll_area,
    input_bar,
    restart_button
], layout=Layout(
    width="100%",
    height="460px",
    display="flex",
    flex_flow="column",
    border="1px solid #ccc"
))


## 📄 Step 6: Document Upload & Analysis Interface
### 🔧 What This Cell Does:
This step extends the EmotiBot UI by adding a document selection and analysis control for users to upload .pdf or .txt files from the Kaggle environment.

#### * **File Detection:**
 * Dynamically scans the **/kaggle** directory to populate a dropdown with valid document files.

#### * **How to Upload the Files/Documents:**
 * Go to Kaggle UI Goto **File->Upload input->Upload DataSet and select files you want**.

#### * **Dropdown Selector:**
 * Users can choose one document from the list for emotional and semantic analysis.

#### * **Analyze Button:**
 * Triggers the document processing logic—extracting text, summarizing content, and identifying emotional tone.

#### * **UI Integration:**
  * Appends the dropdown and button to the existing chat layout (chat_ui) for seamless visual flow.



In [ ]:
from ipywidgets import Dropdown, HBox

def list_kaggle_docs():
    paths = []
    for root, _, files in os.walk("/kaggle"):
        for file in files:
            if file.endswith(".pdf") or file.endswith(".txt"):
                print(file)
                paths.append(os.path.join(root, file))
                
    return paths or [("No file uploaded", "")]

file_dropdown = Dropdown(
    options=list_kaggle_docs(),
    description='📄 File:',
    layout=Layout(width="70%")
)

analyze_button = Button(
    description="Analyze Document",
    icon="search",
    layout=Layout(
        width="180px",
        height="30px",            
    ),
    style=dict(
        button_color="#258a06",       
        border="1px solid #ccc",
        text_color="white",           
        font_weight="500",
        font_size="13px"
    ),
    tooltip="Run document analysis and emotional summary"
)


# Add to UI
chat_ui.children += (HBox([file_dropdown, analyze_button]),)

print("UI components are ready...")

## ⚙️ Step 7: 🖼️ Render Chat Messages with Avatars, Animations & States
### 🔧 What This Cell Does:
This step defines the render_messages() function, which converts the chat history into styled HTML to visually display the conversation between the user and EmotiBot.

#### * **Message Types Rendered:**
   * ✍️ ***Typing Indicator:*** Shows "EmotiBot is typing…" animation.
   * 🧠 ***Document Analysis:*** Displays "Analyzing your document…" while background processing runs.
   * 📊 ***Session Summary:*** Indicates that the emotional summary is being prepared.
   * 💬 ***AI Messages:*** Formats replies from EmotiBot with a soft bubble style and avatar.
   * 👤 ***User Messages:*** Shows user replies aligned to the right with a distinct color scheme.
   * 📄 ***Document Uploads:*** Confirms when a user uploads a file.

#### * **UI Styling:**
 * Rounded avatars and chat bubbles.
 * Consistent padding, box shadows, and color-coded responses.
 * Uses flexible HTML rendering to support emojis, bold text, or structured replies.



In [ ]:
def render_messages(history):
    html = ""
    for msg in history:
        if msg == "__typing__":
           html += f"""
            <div style='display:flex; align-items:flex-start; margin-bottom:10px;'>
                <img src="{BOT_AVATAR}" style="width:36px; height:36px; border-radius:50%; margin-right:10px;border:1px solid #ccc;">
                <div style="background:#ffffff; color:#000; padding:10px 14px; border-radius:10px;
                            box-shadow:0 1px 4px rgba(0,0,0,0.1); max-width:70%;border: 1px solid #ccc;">
                    <div style="font-style:italic;">
                        EmotiBot is typing...
                    </div>
                </div>
            </div>
            """
        elif msg == "__analyzing__":
            html += f"""
            <div style='display:flex; align-items:flex-start; margin-bottom:10px;'>
                <img src="{BOT_AVATAR}" style="width:36px; height:36px; border-radius:50%; margin-right:10px;border:1px solid #ccc;">
                <div style="background:#ffffff; color:#000; padding:10px 14px; border-radius:10px;
                            box-shadow:0 1px 4px rgba(0,0,0,0.1); max-width:70%;border: 1px solid #ccc;">
                    <div style="font-style:italic;">
                        EmotiBot is analyzing your document...
                    </div>
                </div>
            </div>
            """ 
        elif msg == "__preparing_summary__":
            html += f"""
            <div style='display:flex; align-items:flex-start; margin-bottom:10px;'>
                <img src="{BOT_AVATAR}" style="width:36px; height:36px; border-radius:50%; margin-right:10px;border:1px solid #ccc;">
                <div style="background:#ffffff; color:#000; padding:10px 14px; border-radius:10px;
                            box-shadow:0 1px 4px rgba(0,0,0,0.1); max-width:70%;border: 1px solid #ccc;">
                    <div style="font-style:italic;">
                        EmotiBot is preparing your session summary...
                    </div>
                </div>
            </div>
            """              
        elif isinstance(msg, AIMessage):
           html += f"""
            <div style='display:flex; align-items:flex-start; margin-bottom:10px;'>
                <img src="{BOT_AVATAR}" style="width:36px; height:36px; border-radius:50%; margin-right:10px;border:1px solid #ccc;">
                <div style="background:#ffffff; color:#000; padding:10px 14px; border-radius:10px;
                            box-shadow:0 1px 4px rgba(0,0,0,0.1); max-width:70%;border: 1px solid #ccc;">
                    <div style='font-weight:600;'>EmotiBot</div>
                    <div>{msg.content}</div>
                </div>
            </div>
            """
        elif isinstance(msg, HumanMessage):
            html += f"""
            <div style='display:flex; align-items:flex-start; justify-content:flex-end; margin-bottom:10px;'>
                <div style="background:#014f86; color:white; padding:10px 14px; border-radius:10px;
                            max-width:70%; box-shadow:0 1px 4px rgba(0,0,0,0.1); text-align:left;">
                    <div style='font-weight:600;'>You</div>
                    <div>{msg.content}</div>
                </div>
                <img src="{USER_AVATAR}" style="width:36px; height:36px; border-radius:50%; margin-left:10px;">
            </div>
            """
        elif isinstance(msg, dict) and msg.get("type") == "file_upload":
            html += f"""
            <div style='display:flex; align-items:flex-start; justify-content:flex-end; margin-bottom:10px;'>
                <div style="background:#014f86; color:white; padding:10px 14px; border-radius:10px;
                            max-width:70%; box-shadow:0 1px 4px rgba(0,0,0,0.1); text-align:left;">
                    <div style='font-weight:600;'>📄 Document Uploaded</div>
                    <div>{msg.get("filename", "Unknown")}</div>
                </div>
                <img src="{USER_AVATAR}" style="width:36px; height:36px; border-radius:50%; margin-left:10px;">
            </div>
            """            
    return html

## ⚙️ Step 8: 🔄 Render Chat + Auto-Scroll to Latest Message
### 🔧 What This Cell Does:
This step defines two utility functions to refresh the chat interface and auto-scroll to the latest message:

* **render_chat():**
    * Calls the render_messages() function to update the chat display with the latest message history.
    * Ensures that all new messages, indicators, or summaries are instantly visible in the UI.

  

* **scroll_to_bottom():**
    * Injects a small JavaScript snippet into the notebook to scroll the message area to the bottom.
    * Ensures that the user always sees the most recent message without needing to scroll manually.



In [ ]:
def render_chat():
    chat_html.value = render_messages(chat_history)
    scroll_to_bottom()
    
def scroll_to_bottom():
    display(DHTML("""
    <script>
    setTimeout(() => {
        const el = document.querySelector('.widget-html-content');
        if (el) el.scrollTop = el.scrollHeight;
    }, 100);
    </script>
    """))


## 📄 Step 9: Document Understanding, Structured output/JSON mode and Use of Long context window
### 🔧 What This Cell Does:
This step handles the backend logic of document analysis after a user selects a file and clicks the *"Analyze"* button.

* **Reads the document:**
    * Supports .pdf and .txt files using PyPDF2 and standard file reading.
    * Limits the input text to ~6000 characters for efficient GenAI processing.

* **Triggers Gemini 2.0 Flash:**
    * Asks the model to summarize the document empathetically.
    * Then extracts emotional tone in structured JSON form (e.g., {"stress": 60, "hope": 40}).

* **Stores Results:**
    * Saves the filename, emotional profile, and summary into uploaded_docs.
    * Marks documents as processed to avoid duplicate work.

* **Updates Chat Interface:**
    * Simulates typing/analysis indicators.
    * Adds a confirmation message once the analysis is done.U
    * Uses threading to run the analysis asynchronously while keeping UI responsive.

### 🤖 GenAI Features Demonstrated:
* **Document Understanding:**
    * Extracts and processes raw text from structured documents using Gemini, enabling deeper insights.
* **Structured Output / JSON Mode:**
    * The model is prompted to return emotional data in a clean JSON format, which is parsed for later visulization
* **Long Context Window:**
    * Up to 6000 characters of text in one go, allowing accurate understanding of full documents.

In [ ]:
from PyPDF2 import PdfReader
import re, base64, matplotlib.pyplot as plt
from io import BytesIO
import threading
import time

def handle_kaggle_file(_=None):
    global uploaded_docs, analyzed_files_seat, currently_processing_file

    path = file_dropdown.value
    if not path or not os.path.exists(path):
        chat_history.append(AIMessage(content="⚠️ Please select a valid file."))
        render_chat()
        return

    filename = os.path.basename(path)

    # Check if it's already uploaded or in-progress
    if filename in analyzed_files_set or filename in currently_processing_file:
        return  # Skip silently — already done or in progress

    # Add to processing set to lock
    currently_processing_file.add(filename)

    # Simulate user message
    chat_history.append(HumanMessage(content=f"📄 Document Uploaded: {filename}"))
    render_chat()

    # Analyzing document
    chat_history.append("__analyzing__")
    render_chat()

    def analyze():
        try:
            if filename.endswith(".pdf"):
                reader = PdfReader(path)
                text = "\n".join([p.extract_text() or "" for p in reader.pages])
            elif filename.endswith(".txt"):
                with open(path, "r", encoding="utf-8") as f:
                    text = f.read()
            else:
                insert_bot_message("❌ Unsupported format. Use PDF or TXT.")
                currently_processing_file.discard(filename)
                return
        except Exception as e:
            insert_bot_message(f"❌ Could not read file: {e}")
            currently_processing_file.discard(filename)
            return

        #------------------- Use of Long Context Window-----------------#
        
        text = text[:6000]
        
        #---------------------------------------------------------------#
        
        model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

        # Summary
        summary_prompt = f"Summarize this document empathetically (under 150 words):\n\n{text}"
        summary_result = model.invoke([HumanMessage(content=summary_prompt)])
        summary_text = summary_result.content.strip()

        # Emotions
        emotion_prompt = (
            "Extract emotional tone from this document. Return JSON with emotions as keys and percentages (must total 100):\n\n"
            + text
        )
        emotion_result = model.invoke([HumanMessage(content=emotion_prompt)])
        match = re.search(r"\{.*?\}", emotion_result.content, re.DOTALL)
        emotion_data = eval(match.group()) if match else {}
        
        uploaded_docs.append({
            "filename": filename,
            "summary": summary_text,
            "emotions": emotion_data
        })
        
        analyzed_files_set.add(filename)
        currently_processing_file.discard(filename)

        time.sleep(1.5)  # delay
        bot_msg = f"<b>📄 {filename} Analyzed successfully.</b>"
        insert_bot_message(bot_msg)

    threading.Thread(target=analyze).start()


def insert_bot_message(bot_message_html):
    for i in reversed(range(len(chat_history))):
        if chat_history[i] == "__analyzing__":
            chat_history[i] = AIMessage(content=bot_message_html)
            break
    render_chat()


## 💬 Step 10: Send Message to EmotiBot + Handle Live Typing, Graunding with EMOTIBOT_SYSINT
### 🔧 What This Step Does:
This step powers the live chat experience between the user and EmotiBot:

* **Captures User Input:**
    * Reads text typed into the input field.
    * Sends it as a HumanMessage to the chat history.
   
* **Shows Typing Indicator:**
    * Temporarily displays "EmotiBot is typing..." in the chat UI while the response is being generated.

* **Invokes Gemini via LangGraph:**
    * Calls the chat_graph_ui flow with the user’s message and system instructions (EMOTIBOT_SYSINT).
    * Receives the assistant’s emotionally grounded reply.

* **Updates UI with Response:**
    * Replaces the typing indicator with the final response from Gemini.
    * Uses threading to simulate realistic delay and prevent UI freezing.

* **Keyboard Support:**
    * on_key_press() allows pressing Enter to send messages, enhancing UX.

### 🤖 GenAI Capabilities Demonstrated:

#### Grounding is here in the model’s reply is based on:
   * The user’s actual message.
   * The system prompt (EMOTIBOT_SYSINT) defined earlier.
   * Instead of a generic reply, Gemini’s response is:
    **“Grounded” in the conversation—emotionally relevant and context-aware.**
   * This improves trust, coherence, and emotional accuracy.

In [ ]:
import asyncio
import threading
from langchain_core.messages import HumanMessage, AIMessage
from IPython.display import display, clear_output, HTML

def handle_send(_=None):
    user_msg = text_input.value.strip()
    if not user_msg:
        return
        
    text_input.value = ""

    # Add user message
    chat_history.append(HumanMessage(content=user_msg))
    render_chat()

    # Add EmotiBot typing indicator
    chat_history.append("__typing__")
    render_chat()

    # Get EmotiBot response in background
    def fetch_and_update():
        import time
        time.sleep(1.5)  # simulate delay
        state = chat_graph_ui.invoke({"messages": [("user", user_msg)]})
        bot_reply = state["messages"][-1]

        # Replace __typing__ with actual message
        for i in reversed(range(len(chat_history))):
            if chat_history[i] == "__typing__":
                chat_history[i] = bot_reply
                break

        render_chat()

    threading.Thread(target=fetch_and_update).start()

def on_key_press(change):
    if change["new"].endswith("\n"):
        text_input.value = text_input.value.strip()
        handle_send()

## 🔚Step 11: End Session & Generate Emotional Summary + Reflection Suggestions Using Few Short Prompt
### 🔧 What This Step Does:
This step handles what happens when the user clicks “End Chat”. It uses Gemini to:
1. Analyze the entire chat conversation for emotional tone and summarize it supportively.
2. Combine document emotions (if any) with chat emotions.
3. Generate an emotional pie chart based on the full session.
4. Ask Gemini to suggest 2–3 kind, actionable reflections tailored to the emotional profile.
5. Display a final session summary, including:
   * Emotional chart
   * Chat summary
   * Document summaries
   * Suggested reflections

It uses ***threading*** to run smoothly in the background and prevents repeated execution using flags.

### 🤖 GenAI Capabilities Demonstrated:
1. 🧾 **Structured Output / JSON Mode:** Emotions extracted from chat are returned as JSON-like objects ({"hope": 30, "stress": 70}) for further use.
2. 🧠**Few-shot Prompting:**
    * Uses five rich examples to help the model generate realistic, grounded reflection suggestions.
    * This handles the emotions are formatted ({"emotion": %, ...}).
    * How to respond with compassionate, bullet-point suggestions.
    * The style, tone, and number of outputs expected      

In [ ]:
import json

def on_end_chat(_=None):
    global already_warned_user, session_ended, has_ended_message

    if session_ended:
        return  # Hard lock

    session_ended = True  # Lock execution

    model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

    user_msgs = [msg for msg in chat_history if isinstance(msg, HumanMessage)]
    chat_text = "\n".join([m.content for m in user_msgs]) if user_msgs else ""

    chat_emotions = {}
    combined_emotions = {}
    chat_summary_text = ""
    document_summary_text = ""
    emotion_chart_html = ""

    # Simulate user message ONCE
    if not has_ended_message:
        chat_history.append(HumanMessage(content="🔚 End Chat"))
        has_ended_message = True
        render_chat()

    # Hide UI
    input_bar.layout.display = "none"
    file_dropdown.layout.display = "none"
    analyze_button.layout.display = "none"
    end_chat_button.disabled = True

    # Show "preparing..." only once
    if "__preparing_summary__" not in chat_history:
        chat_history.append("__preparing_summary__")
        render_chat()

    def summarize_and_display():
        nonlocal chat_emotions, combined_emotions, chat_summary_text, document_summary_text
    
        if not chat_text.strip() and not analyzed_files_set:
            time.sleep(1.5)
            insert_summary_message("⚠️ No valid conversation or analyzed document found.")
            restart_button.layout.display = "flex"
            return
    
        try:
            #----------------------- Extract chat emotions using Structured output/JSON mode-----------------------
            if chat_text.strip():
                emotion_prompt = (
                    "Extract emotions from this chat and return JSON with emotion names as keys and percentages (must total 100):\n\n"
                    + chat_text
                )
                emotion_result = model.invoke([HumanMessage(content=emotion_prompt)])
                match = re.search(r"\{.*?\}", emotion_result.content, re.DOTALL)
                if match:
                    try:
                        chat_emotions = eval(match.group())
                    except:
                        chat_emotions = {}
    
                # Generate chat summary
                summary_prompt = (
                    "Summarize this chat session in a supportive tone (under 150 words):\n\n"
                    + chat_text
                )
                summary_result = model.invoke([HumanMessage(content=summary_prompt)])
                chat_summary_text = summary_result.content.strip()
    
            # Gather document summaries and emotions
            for doc in uploaded_docs:
                if doc["filename"] in analyzed_files_set:
                    document_summary_text += f"<b>📄 {doc['filename']}:</b> {doc['summary']}<br><br>"
                    for emo, val in doc["emotions"].items():
                        combined_emotions[emo] = combined_emotions.get(emo, 0) + val
    
            # Merge chat + doc emotions
            for emo, val in chat_emotions.items():
                combined_emotions[emo] = combined_emotions.get(emo, 0) + val
    
            # Normalize totals
            total = sum(combined_emotions.values())
            if total > 0:
                combined_emotions = {k: round(v / total * 100, 1) for k, v in combined_emotions.items()}
                fig, ax = plt.subplots()
                ax.pie(combined_emotions.values(), labels=combined_emotions.keys(), autopct="%1.1f%%", startangle=90)
                ax.set_title("🧠 Session Emotions")
                buf = BytesIO()
                fig.savefig(buf, format="png")
                buf.seek(0)
                chart_base64 = base64.b64encode(buf.read()).decode()
                plt.close(fig)
                emotion_chart_html = f"<img src='data:image/png;base64,{chart_base64}' width='350'/>"
            else:
                emotion_chart_html = "<i>(No emotional data found.)</i>"

            #--------------Few-short Propting-------------------------#
            
            # Combined suggestion block Using few-short prompting
            
            few_shot_combined_prompt = """Given this emotional profile (from both chat and documents), 
            suggest 2–3 supportive, realistic, and kind actions someone might take. Do not include any JSON, 
            percentages, or explanation—just list the suggestions in bullet point format.
            EXAMPLE:
            Emotions: {"anxiety": 50, "self-doubt": 30, "hope": 20}
            Suggestions:
            - Remind yourself that growth isn’t linear—progress is still progress.
            - Take a few moments to breathe deeply and be present.
            - Try writing down what feels heaviest, then let it go.
            
            EXAMPLE:
            Emotions: {"stress": 60, "burnout": 30, "restlessness": 10}
            Suggestions:
            - Take 10 minutes away from your screen and just breathe.
            - Prioritize one simple task today—let that be enough.
            - Give yourself permission to rest without guilt.
            
            EXAMPLE:
            Emotions: {"confusion": 40, "overwhelm": 40, "focus": 20}
            Suggestions:
            - Break things down into smaller, manageable pieces.
            - Remind yourself it's okay not to have all the answers yet.
            - Try writing down what you *do* know—it brings clarity.
            
            EXAMPLE:
            Emotions: {"grief": 45, "hope": 30, "love": 25}
            Suggestions:
            - It's okay to cry or feel heavy—honor those feelings.
            - Reach out to someone and simply talk or share a memory.
            - Do one gentle thing today that feels comforting.
            
            EXAMPLE:
            Emotions: {"anger": 50, "frustration": 30, "calm": 20}
            Suggestions:
            - Step away and take three deep breaths before responding.
            - Go for a short walk or listen to calming music.
            - Let yourself write (even if messy) what you wish you could say.
            
            NOW ANALYZE:
            Emotions: """ + str(combined_emotions)

    
            suggestion_result = model.invoke([HumanMessage(content=few_shot_combined_prompt)])
            
            combined_suggestions = suggestion_result.content.strip()

            combined_suggestions_lines = [
                line.strip().lstrip("-• ") for line in combined_suggestions.splitlines() if line.strip()
            ][:3]
            
            # Reformat as bullet list
            combined_suggestions = "<br>".join(f"- {line}" for line in combined_suggestions_lines)


    
            # Final safe HTML output
            final_html = f"""
            <b>🧠 Emotional Profile(Chat+Document's):</b><br>{emotion_chart_html}<br><br>
            {f"<b>💬 Chat Summary:</b><br>{chat_summary_text}<br><br>" if chat_summary_text else ""}
            {f"<b>📄 Document Summaries:</b><br>{document_summary_text}" if document_summary_text else ""}
            {f"<br><br><b>🌱 Suggested Reflections:</b><br><ul style='margin-top:8px;'>{''.join(f'<li>{s[2:]}</li>' for s in combined_suggestions.split('<br>'))}</ul>" if combined_suggestions else ""}
            """
    
            time.sleep(1.5)
            insert_summary_message(final_html)
    
        except Exception as e:
            insert_summary_message(f"<b>❌ Error occurred:</b><pre>{str(e)}</pre>")
    
        restart_button.layout.display = "flex"


    threading.Thread(target=summarize_and_display).start()

    text_input.value = ""

## 🔁 Step 12: Finalize UI Events, Restart Logic, and Launch EmotiBot Interface
### 🔧 What This Step Does:
This final step connects all the buttons to their actions and brings the EmotiBot chatbot to life in the notebook:

* 🔚 **End Chat button** triggers emotional summary and insights *(on_end_chat)*
* 📄 **Analyze Document button** invokes Gemini-powered document understanding *(handle_kaggle_file)*
* 🔄 **Restart Session** clears state and starts a new chat *(restart_chat)*
* 📤 **Send Button** and Enter key trigger live message handling *(handle_send)*
* 🖥️ **Displays** the final chat UI using display *(chat_ui)*

**It also:**
   * Clears the previous session data.
   * Resets all flags and UI elements.
   * Adds Gemini’s initial friendly greeting from EmotiBot.

### 🧠 So How Is GenAI Evaluation Used in this Step (lightweight):

#### Step 12 enables GenAI Evaluation by connecting the “End Chat” button to the on_end_chat() function.
This function calls Gemini to:

* Analyze emotional tone from the full conversation.
* Generate a supportive summary.
* Visualize emotions in a pie chart.
* Suggest next-step reflections based on emotional insights

In [ ]:
def insert_summary_message(html):
    for i in reversed(range(len(chat_history))):
        if chat_history[i] == "__preparing_summary__":
            chat_history[i] = AIMessage(content=html)
            render_chat()
            return
    chat_history.append(AIMessage(content=html))  # fallback
    render_chat()


    # Fallback if not found
    chat_history.append(AIMessage(content=html))
    render_chat()


def restart_chat(_=None):
    global already_warned_user, session_ended, has_ended_message

    chat_history.clear()
    uploaded_docs.clear()
    analyzed_files_set.clear()
    currently_processing_file.clear()

    already_warned_user = False
    session_ended = False
    has_ended_message = False  # Reset end message flag

    chat_html.value = ""
    input_bar.layout.display = "flex"
    file_dropdown.layout.display = "block"
    analyze_button.layout.display = "block"
    restart_button.layout.display = "none"
    end_chat_button.disabled = False

    chat_history.append(AIMessage(content="👋 Hi again! I'm EmotiBot. How can I support you today?"))
    render_chat()

# Global storage for doc analysis
uploaded_doc_summary_html = ""
uploaded_doc_emotions = {}

# Button setup
end_chat_button.on_click(on_end_chat)

analyze_button.on_click(handle_kaggle_file)


restart_button.on_click(restart_chat)


# Reset chat
chat_history.clear()
chat_html.value = ""

# Set up events
send_button.on_click(handle_send)
text_input.observe(on_key_press, names="value")
display(chat_ui)


## 🕹️ How to Use EmotiBot
### 1️⃣ Start Chat
Type your thoughts in the box and hit  **➤ Send**. EmotiBot replies with emotional support.

### 2️⃣ Upload Document
Choose a **.pdf or .txt** from the 📄 **File Dropdown**, then click  **🔍 Analyze Document** it analyze document and give the summary and 📈 pie chart of emotions after the end of chat.

### 3️⃣ End Session
Click End Chat to get a 💬 summary, 📈 emotion pie chart, and 🌱 self-care suggestions.

### 4️⃣ Restart Anytime
Click **🔄 Restart Session** to clear chat and begin again.

✅ Use it to reflect, analyze writing, or explore emotional tone in conversations & documents.